In [5]:
import logging

logging.basicConfig(level=logging.INFO)


In [6]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss

# from pennylane import NesterovMomentumOptimizer

from qcc.ml.data import Data, BinaryData
from qcc.ml.optimize import Optimizer
from qcc.ml.data import ImageTransform, ImageTransform1D, ClassicalImageTransform
from qcc.experiment import Experiment
from qcc.ml.model import Model

from pathlib import Path
from qcc.file import new_dir

from qcc.quantum.pennylane.ansatz import MQCCOptimized # as Module
# from qcc.quantum.pennylane.ansatz import QCNN as Module
# from qcc.ml.cnn import ConvolutionalNeuralNetwork as Module
# from qcc.ml.mlp import MultiLayerPerceptron as Module
# from qcc.ml.quanvolution import QuanvolutionalNeuralNetwork as Module
from qcc.ml.quantum import MQCCNonHybrid as Module

# from qcc.quantum.pennylane.ansatz.basic import BasicFiltering6
from qcc.quantum.pennylane.pyramid import Pyramid
from qcc.quantum.pennylane.c2q import ConvolutionAngleFilter, ConvolutionComplexAngleFilter, ConvolutionFilter
from qcc.quantum.pennylane.local import define_filter
from qcc.ml.quantum import AnsatzFilter


In [7]:
# Meta parameters
name = "sdfljhvosdv"
filename = Path(f"results/{name}")
num_trials = 1
silent = False
is_quantum = False

# Ansatz parameters
dims = (16, 16, 1)
num_layers = 3
module_options = {
    "U_kernel": ConvolutionAngleFilter,
    # "pre_op": True,
    "num_features": 4,
    # "U_fully_connected": ConvolutionAngleFilter,
    # "pooling": True,
    # "kernel_shape": (2, 2),
    # "q2c_method": "parity"
    # "bias": False
    "ansatz": MQCCOptimized
}


In [8]:
filename = new_dir(filename, overwrite=True)
filename = filename / name

if is_quantum:
    module = Module.from_dims
else:
    module = Module

# Create module
module = module(
    dims,
    num_layers=num_layers,
    **module_options,
)
data = BinaryData(
    FashionMNIST,
    ImageTransform1D(dims) if is_quantum else ClassicalImageTransform(dims),
    batch_size=(8,8),
)
optimizer = Optimizer(Adam)
loss = CrossEntropyLoss()
model = Model.with_logging(module, data, optimizer, loss, epoch=8)

# Log important values
model.logger.info(f"Circuit ID: {name}")
model.logger.info(f"{module=}")
model.logger.info(f"{data=}")
model.logger.info(f"{optimizer=}")
model.logger.info(f"{loss=}")

model.logger.info(f"{num_trials=}")
model.logger.info(f"{dims=}")
model.logger.info(f"{num_layers=}")
# model.logger.info(f"{1=}")
model.logger.info(f"{module_options=}")

# Save circuit drawing
if is_quantum:
    draw_path = filename.with_stem(f"{name}_circuit")
    module.draw(filename=draw_path, decompose=True)


INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 21
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 78
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 20
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 68
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 19
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 62
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 33
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 131
2023-12-14 13:20:33,826: (mqccnonhybrid) Circuit ID: sdfljhvosdv
2023-12-14 13:20:33,827: (mqccnonhybrid) module=MQCCNonHybrid(
  (0): MQCCLayer(
    (mqcc): MQCCOptimized()
  )
  (1): MQCCLayer(
    (mqcc): MQCCOptimized()
  )
  (2): MQCCLayer(
    (mqcc): MQCCOptimized()
  )
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): FullyConnectedLayer(
    (fc): FullyConnected()
  )
)
2023-12-14 13:20:33,827: (mqccnonhybrid) data=BinaryData(dataset=<class 'torchvision.datasets.mnist.FashionMNIST'>, transform=image_transform_classical, target_transform=None, batch_size=(8, 8

In [9]:
# Run experiment
results_schema = ["accuracy", "training_time", "testing_time"]
experiment = Experiment(model, num_trials, results_schema)
# experiment.partial(silent=silent)
results = experiment(filename=filename)


100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%
2023-12-14 13:21:09,545: (mqccnonhybrid) Number of Parameters: 82
/Users/mingyoungjeng/Documents/repos.nosync/quantum-convolutional-classifier/src/qcc/quantum/quantum.py:135: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3618.)
  data = data.reshape(size[::-1]).T


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



2023-12-14 13:23:20,119: (mqccnonhybrid) (Epoch 1) Training took 128.06920 sec
2023-12-14 13:23:35,729: (mqccnonhybrid) (Epoch 1) Testing took: 15.38113 sec
2023-12-14 13:23:35,729: (mqccnonhybrid) (Epoch 1) Accuracy: 95.600%


KeyboardInterrupt: 

In [ ]:
# Print accuracy results
metrics = ("median", "mean", "max", "min", "std")
for name in results.columns:
    col = results[name]
    msg = (f"{metric}={getattr(col, metric)()}" for metric in metrics)
    msg = ", ".join(msg)
    msg = f"{name}: {msg}"
    model.logger.info(msg)

# Save aggregated loss history figure
display(experiment.draw(filename))
